# RDF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/rudof-project/tutorials/blob/main/rudof/rdf.ipynb)

This document contains a short introduction to [RDF](https://www.w3.org/TR/rdf11-primer/) using [rudof](https://rudof-project.github.io/).


## Preliminaries: install and configure rudof

The library is available as `pyrudof`.

In [2]:
!pip install pyrudof

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 62.4 MB/s eta 0:00:00


The main entry point if a class called [`Rudof`](https://pyrudof.readthedocs.io/en/stable/library.html#rudof) through which most of the functionality is provided.

In [3]:
from pyrudof import Rudof, RudofConfig

In order to initialize that class, it is possible to pass a [`RudofConfig`](https://pyrudof.readthedocs.io/en/stable/library.html#pyrudof.RudofConfig) instance which contains configuration parameters for customization. An instance of `RudofConfig` can be obtained from the default initialization method or can be read from a TOML file.

In [4]:
rudof = Rudof(RudofConfig())

## RDF data model

RDF is based on statements or triples of the form `<subject> <predicate> <object>` where the predicates are identified by IRIs and in the most basic form, subjects and objects are also IRIs. An example could be: `<http://example.org/alice> <http://example.org/knows> <http:/example.org/bob>`. In `rudof`, it is possible to load a set of triples with as follows:

In [5]:
rudof.read_data_str("<http://example.org/alice> <http://example.org/knows> <http://example.org/bob> .")

## Handling RDF data

In [6]:
rudof.read_data_str("""
prefix : <http://example.org/>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:x a :Person     ;
   :name "Alice" ;
   :knows :y     .
:y a :Person     ;
   :name "Bob"   ;
   :knows :z     .
:z a :Person     ;
   :knows :x     .
""")

The RDF data can easily be merged with other data.

In [7]:
rudof.read_data_str("""
prefix : <http://example.org/>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:u a :Person     ;
   :name "Dave" ;
   :knows :y     .
:v a :Person     ;
   :name "Emily" ;
   :knows _:1    .
_:1 a :Person     ;
   :knows :x     .
""")

And the data can be serialized to different formats.

In [8]:
from pyrudof import RDFFormat
str = rudof.serialize_data(format=RDFFormat.NTriples)
print(str)

<http://example.org/x> <http://example.org/name> "Alice" .
<http://example.org/x> <http://example.org/knows> <http://example.org/y> .
<http://example.org/x> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Person> .
<http://example.org/v> <http://example.org/name> "Emily" .
<http://example.org/v> <http://example.org/knows> _:1 .
<http://example.org/v> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Person> .
<http://example.org/y> <http://example.org/name> "Bob" .
<http://example.org/y> <http://example.org/knows> <http://example.org/z> .
<http://example.org/y> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Person> .
<http://example.org/alice> <http://example.org/knows> <http://example.org/bob> .
<http://example.org/u> <http://example.org/name> "Dave" .
<http://example.org/u> <http://example.org/knows> <http://example.org/y> .
<http://example.org/u> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.org/Per

## Running SPARQL queries

In [48]:
from pyrudof import Rudof, RudofConfig, QuerySolutions
rudof = Rudof(RudofConfig())
rudof.reset_all()

In [49]:
rdf = """
prefix :    <http://example.org/>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:alice a :Person ;
 :name      "Alice"                ;
 :birthDate "2005-03-01"^^xsd:date ;
 :worksFor  :acme                   .
:bob a :Person   ;
 :name      "Robert Smith"         ;
 :birthDate "2003-01-02"^^xsd:date ;
 :worksFor  :acme  .
:acme a :Company ;
 :name "Acme Inc." .
"""
rudof.read_data_str(rdf)

In [50]:
query = """
PREFIX : <http://example.org/>

SELECT ?person ?name ?date ?company WHERE {
  ?person a          :Person ;
          :name      ?name   ;
          :birthDate ?date   ;
          :worksFor  ?c   .
  ?c      :name      ?company .
}
"""

results = rudof.run_query_str(query)


Show the results:

In [51]:
print(results.show())

╭───┬─────────┬────────────────┬────────────────────────┬─────────────╮
│   │ ?person │ ?name          │ ?date                  │ ?company    │
├───┼─────────┼────────────────┼────────────────────────┼─────────────┤
│ 1 │ :alice  │ "Alice"        │ "2005-03-01"^^xsd:date │ "Acme Inc." │
├───┼─────────┼────────────────┼────────────────────────┼─────────────┤
│ 2 │ :bob    │ "Robert Smith" │ "2003-01-02"^^xsd:date │ "Acme Inc." │
╰───┴─────────┴────────────────┴────────────────────────┴─────────────╯

